In [1]:
# import pandas
import pandas as pd

# import plotly
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
#%%  Reading PKL file into a DataFrame

ethereum_tx = pd.read_pickle('files\\ethereum_tx.pkl')
bitcoin_tx = pd.read_pickle('files\\bitcoin_tx.pkl')
binance_tx = pd.read_pickle('files\\binance_tx.pkl')

# assume both have the same columns: ['chain','date','tx_count']
web3_tx = pd.concat(
    [ethereum_tx, bitcoin_tx, binance_tx],
    ignore_index=True
)

# Convert date to datetime and chain to categorical
web3_tx['chain'] = pd.Categorical(web3_tx['chain'])
web3_tx = web3_tx.sort_values(['chain','date']).reset_index(drop=True)

# group by chain + week, summing tx_count
web3_tx = (web3_tx.groupby(['chain',pd.Grouper(key='date', freq='W')])['tx_count'].sum().reset_index(name='tx_count'))

print(web3_tx.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1152 entries, 0 to 1151
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   chain     1152 non-null   category      
 1   date      1152 non-null   datetime64[ns]
 2   tx_count  1152 non-null   int64         
dtypes: category(1), datetime64[ns](1), int64(1)
memory usage: 19.4 KB
None


C:\Users\Matheus\AppData\Local\Temp\ipykernel_34592\4162434964.py:20: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby([


In [10]:
# save the DataFrame to a pickle file
web3_tx = pd.DataFrame.to_pickle(web3_tx,'files\web3_tx.pkl')

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Matheus\AppData\Local\Temp\ipykernel_34592\2049700646.py:2: SyntaxWarning: invalid escape sequence '\w'
  web3_tx = pd.DataFrame.to_pickle(web3_tx,'files\web3_tx.pkl')


In [14]:
# Create the figure
fig1 = px.line(
    web3_tx,
    x='date',
    y='tx_count',
    color='chain',
    custom_data=['chain', 'tx_count'],
    title='Blockchain Weekly Activity',
    labels={
        'date': 'Date',
        'tx_count': 'Transactions',
        'chain': 'Blockchain',
    },
    color_discrete_map={
        'bitcoin': '#F7931A',  # Bitcoin Orange
        'ethereum': '#636BFF',  # Ethereum Blue
        'binance': '#F0B90B',  # Binance Yellow
    },
    markers=False,
    line_shape='spline',
)

# Update traces to reduce marker size
fig1.update_traces(
    line=dict(width= 1.5, smoothing=0.5),
    hovertemplate="<b>%{customdata[0]}</b><br>" +
                  "Transactions: %{customdata[1]:,.2f}<br>" +
                  "<extra></extra>"
)

fig1.update_layout(
    template="plotly_dark",              # inherit base dark theme 
        hovermode="x unified",
        title=dict(font=dict(size=14, color='white'), xanchor="left"),
        font=dict(family="IBM Plex Sans", color="white"),
        legend=dict(yanchor="top", y=0.99, xanchor="left", x=1.05),
        margin=dict(l=10, r=10, t=40, b=10),
        yaxis=dict(title="Transactions"),
        height=500, width=705,
)

# Show the plot
fig1.show(config={
    'displayModeBar': True,
    'displaylogo': False,
    'modeBarButtonsToAdd': ['fullscreen']
})


# get the full HTML as a string, with Plotly.js from CDN
fig1html = fig1.to_html(include_plotlyjs="cdn", full_html=True)

# inject the Google Fonts link right after <head>
font_link = '<link href="https://fonts.googleapis.com/css2?family=IBM+Plex+Sans&display=swap" rel="stylesheet">'
fig1html = fig1html.replace("<head>", f"<head>\n    {font_link}")

# save the HTML to a file
with open("plots/html/blockchain_activity.html", "w") as f:
    f.write(fig1html)